In [18]:
# import zipfile
# import gdown

# file_id = '1wA0RNEJ8tLJ45DkMKFBj1yvKl9bb9zvX'
# gdown.download(f'https://drive.google.com/uc?id={file_id}', 'RAF_DB.zip', quiet=False)
# with zipfile.ZipFile('RAF_DB.zip', 'r') as zip_ref:
#     zip_ref.extractall('RAF_DB')

In [4]:
import os
import time
import random
import glob
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms, models
import torchvision.transforms.functional as TF
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn import linear_model, model_selection
from torch.optim.lr_scheduler import CosineAnnealingLR, ReduceLROnPlateau
from trainer import train, test
from evaluation.mia import MIA
from model import load_model
from RAF_loader import get_data_loaders
from utils import set_seed
from evaluation.accuracy import calculate_accuracy, calculate_all_accuracies
import warnings
warnings.filterwarnings("ignore")

In [5]:
data_dir = "/home/dasol/Unlearning/RAF_DB/DATASET"
data_loaders = get_data_loaders(data_dir, batch_size=64)

print("=== Size of dataset ===")
print(f"Train dataset size: {len(data_loaders['train'].dataset)}")
print(f"Train (Retain) dataset size: {len(data_loaders['retain_train'].dataset)}")
print(f"Forget (Train) dataset size: {len(data_loaders['forget_train'].dataset)}")
print(f"Validation dataset size: {len(data_loaders['val'].dataset)}")
print(f"Test dataset size: {len(data_loaders['test'].dataset)}")
print(f"Retain Test dataset size: {len(data_loaders['retain_test'].dataset)}")
print(f"Forget Test dataset size: {len(data_loaders['forget_test'].dataset)}")

Train dataset size: 11044
Train (Retain) dataset size: 7730
Forget (Train) dataset size: 3314
Validation dataset size: 1227
Test dataset size: 3068
Retain Test dataset size: 7730
Forget Test dataset size: 3314
=== Size of dataset ===
Train dataset size: 11044
Train (Retain) dataset size: 7730
Forget (Train) dataset size: 3314
Validation dataset size: 1227
Test dataset size: 3068
Retain Test dataset size: 7730
Forget Test dataset size: 3314


## Base Model

### Training Original Model

In [19]:
learning_rate = 0.01
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = load_model(7, device, model_path=None)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4)
scheduler = CosineAnnealingLR(optimizer, T_max=200)

num_epochs = 100
best_val_acc = 0
best_epoch = 0

history = []
accuracy = []
for epoch in range(num_epochs):
    train_loss, train_acc = train(model, data_loaders['train'], criterion, optimizer, scheduler, device, epoch)
    val_loss, val_acc = test(model, data_loaders['val'], criterion, device)
    history.append((train_loss, train_acc))
    accuracy.append((val_loss, val_acc))

    if val_acc > best_val_acc:
        print("[Info] best val accuracy!")
        best_val_acc = val_acc
        best_epoch = epoch
        #torch.save(model.state_dict(), f'checkpoints/best_{epoch + 1}_emotion_resnet_original.pth')

#torch.save(model.state_dict(), f'checkpoints/resnet/last_{num_epochs}_emotion_original.pth')

In [21]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_path =  'checkpoint/last_100_emotion_original.pth'
model = load_model(7, device, model_path)
criterion = nn.CrossEntropyLoss()

test_acc = calculate_accuracy(model, data_loaders['test'], criterion, device)
mia = MIA(model, data_loaders["retain_test"], data_loaders['forget_test'], data_loaders['test'], device)
final_score = (test_acc["Acc"] + 1 - abs(mia["Forgetting Score"] * 2)) / 2
print(f'Test Acc: {test_acc}')
print(f'MIA: {mia}')
print(f'Final score: {final_score}')

Test Acc: {'Loss': 0.018342545638656223, 'Acc': 0.7503259452411994, 'Top-2 Acc': 0.8872229465449805}
MIA: {'MIA Regression Accuracy': 0.6036706349206349, 'MIA CV Accuracy': 0.6741784037558685, 'Forgetting Score': 0.17417840375586846}
Final score: 0.7009845688647313


### Training Retrained Model

In [22]:
learning_rate = 0.1
retrained_model = load_model(7, device, model_path=None)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(retrained_model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4)
scheduler = CosineAnnealingLR(optimizer, T_max=200)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

num_epochs = 100
best_val_acc = 0
best_epoch = 0

history = []
accuracy = []
for epoch in range(num_epochs):
    train_loss, train_acc = train(retrained_model, data_loaders['retain_train'], criterion, optimizer, scheduler, device, epoch)
    val_loss, val_acc = test(retrained_model, data_loaders['val'], criterion, device)
    history.append((train_loss, val_loss))
    accuracy.append((train_acc, val_acc))

    if val_acc > best_val_acc:
        print("[Info] best val accuracy!")
        best_val_acc = val_acc
        best_epoch = epoch
        #torch.save(retrained_model.state_dict(), f'checkpoint/best_{epoch + 1}_emotion_retrained.pth')

#torch.save(retrained_model.state_dict(), f'checkpoint/last_{num_epochs}_emotion_retrained.pth')

In [17]:
model_path = f'checkpoint/last_100_emotion_retrained.pth'
retrained_model = load_model(7, device, model_path)
criterion = nn.CrossEntropyLoss()

test_acc = calculate_accuracy(retrained_model, data_loaders['test'], criterion, device)
mia = MIA(retrained_model, data_loaders["retain_test"], data_loaders['forget_test'], data_loaders['test'], device)
final_score = (test_acc["Acc"] + 1 - abs(mia["Forgetting Score"] * 2)) / 2

print(f'Test Acc: {test_acc}')
print(f'MIA: {mia}')
print(f'Final score: {final_score}')

Test Acc: {'Loss': 0.017435058815382917, 'Acc': 0.719361147327249, 'Top-2 Acc': 0.8503911342894394}
MIA: {'MIA Regression Accuracy': 0.6055839002267573, 'MIA CV Accuracy': 0.5195618153364631, 'Forgetting Score': 0.019561815336463062}
Final score: 0.8401187583271614


## Unlearning Experiments

### Fine-tuning

In [16]:
model_path = 'checkpoint/last_100_emotion_original.pth'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
finetune_model = load_model(7, device, model_path)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(finetune_model.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)
num_epochs=2
scheduler = CosineAnnealingLR(optimizer, T_max=num_epochs)
for epoch in range(num_epochs):
    train_loss, train_acc = train(finetune_model, data_loaders['retain_train'], criterion, optimizer, scheduler, device, epoch)
    val_loss, val_acc = test(finetune_model, data_loaders['val'], criterion, device)

test_acc = calculate_accuracy(finetune_model, data_loaders['test'], criterion, device)
mia = MIA(finetune_model, data_loaders["retain_test"], data_loaders['forget_test'], data_loaders['test'], device)
final_score = (test_acc["Acc"] + 1 - abs(mia["Forgetting Score"] * 2)) / 2
print(f'Test Acc: {test_acc}')
print(f'MIA: {mia}')
print(f'Final score: {final_score}')

[Epoch: 1 - Training]
[Batch: 80] running train loss: 0.0005701056293219153, running train accuracy: 0.991015613079071
train loss: 0.0004860167033466743, accuracy: 0.9922380447387695
elapsed time: 9.167838335037231
[Test]
[Batch: 1] running test loss: 0.02380838245153427, running test accuracy: 0.671875
test loss: 0.02025782687934123, accuracy: 0.7416462898254395
elapsed time: 1.211104393005371
[Epoch: 2 - Training]
[Batch: 80] running train loss: 0.0002695734336612077, running train accuracy: 0.9974609613418579
train loss: 0.00025526780504395977, accuracy: 0.9975420236587524
elapsed time: 9.124073505401611
[Test]
[Batch: 1] running test loss: 0.023482386022806168, running test accuracy: 0.671875
test loss: 0.020534290418259576, accuracy: 0.7424612641334534
elapsed time: 1.1604735851287842
Test Acc: {'Loss': 0.01818828937819779, 'Acc': 0.7516297262059974, 'Top-2 Acc': 0.8895045632333768}
MIA: {'MIA Regression Accuracy': 0.6203939909297053, 'MIA CV Accuracy': 0.6830985915492958, 'Forget

### CF-K

In [23]:
from unlearn.cfk import cfk_train

model_path =  'checkpoint/last_100_emotion_original.pth'
device = torch.device("cuda:0" if torch.cuda.is_available() else "CPU")
criterion = nn.CrossEntropyLoss()
epochs=2
cfk_model = load_model(7, device, model_path)
cfk_model = cfk_train(cfk_model, data_loaders['retain_train'], data_loaders['test'], device, epochs)

test_acc = calculate_accuracy(cfk_model, data_loaders['test'], criterion, device)
mia = MIA(cfk_model, data_loaders["retain_test"], data_loaders['forget_test'], data_loaders['test'], device)
final_score = (test_acc["Acc"] + 1 - abs(mia["Forgetting Score"] * 2)) / 2
print(f'Test Acc: {test_acc}')
print(f'MIA: {mia}')
print(f'Final score: {final_score}')

[Epoch: 1 - Training]
[Batch: 80] running train loss: 0.0005404932484907477, running train accuracy: 0.990234375
train loss: 0.0005690344670765767, accuracy: 0.9900388121604919
elapsed time: 7.364529848098755
[Test]
[Batch: 1] running test loss: 0.031077299267053604, running test accuracy: 0.625
test loss: 0.018655992786653797, accuracy: 0.7513037919998169
elapsed time: 2.8717427253723145
Epoch [1/2] - Train Loss: 0.0006, Train Accuracy: 0.99%
Epoch [1/2] - Test Loss: 0.0187, Test Accuracy: 0.75%
[Epoch: 2 - Training]
[Batch: 80] running train loss: 0.0005941680913110758, running train accuracy: 0.990039050579071
train loss: 0.0005348306739604026, accuracy: 0.991073727607727
elapsed time: 7.329920291900635
[Test]
[Batch: 1] running test loss: 0.0306535754352808, running test accuracy: 0.609375
test loss: 0.018709906634924427, accuracy: 0.7513037919998169
elapsed time: 2.8507800102233887
Epoch [2/2] - Train Loss: 0.0005, Train Accuracy: 0.99%
Epoch [2/2] - Test Loss: 0.0187, Test Accura

### EU-K

In [25]:
from unlearn.euk import euk_unlearn

epochs = 2
model_path =  'checkpoint/last_100_emotion_original.pth'
euk_model = load_model(7, device, model_path)
euk_model = euk_unlearn(euk_model, data_loaders['retain_train'], device, epochs)

test_acc = calculate_accuracy(euk_model, data_loaders['test'], criterion, device)
mia = MIA(euk_model, data_loaders["retain_test"], data_loaders['forget_test'], data_loaders['test'], device)
final_score = (test_acc["Acc"] + 1 - abs(mia["Forgetting Score"] * 2)) / 2
print(f'Test Acc: {test_acc}')
print(f'MIA: {mia}')
print(f'Final score: {final_score}')

Unlearning Epoch [1/2] - Loss: 0.0367, Accuracy: 98.95%
Unlearning Epoch [2/2] - Loss: 0.0356, Accuracy: 98.95%
Test Acc: {'Loss': 0.01820390381166685, 'Acc': 0.7522816166883963, 'Top-2 Acc': 0.8859191655801826}
MIA: {'MIA Regression Accuracy': 0.6033163265306123, 'MIA CV Accuracy': 0.6740219092331767, 'Forgetting Score': 0.17402190923317673}
Final score: 0.7021188991110214


### NegGrad

In [26]:
from unlearn.neg_grad import neggrad_unlearn

class Args:
    def __init__(self):
        self.gpu = 0
        self.lr = 0.001
        self.momentum = 0.9
        self.weight_decay = 5e-4
        self.num_classes = 8
        self.epochs = 1
        self.device = torch.device(f"cuda:{self.gpu}" if torch.cuda.is_available() else "cpu")

args = Args()
criterion = nn.CrossEntropyLoss()
model_path =  'checkpoint/last_100_emotion_original.pth'
neggrad_model = load_model(7, args.device, model_path)
neggrad_unlearn(neggrad_model, data_loaders['forget_train'], criterion, args)

test_acc = calculate_accuracy(neggrad_model, data_loaders['test'], criterion, args.device)
mia = MIA(neggrad_model, data_loaders["retain_test"], data_loaders['forget_test'], data_loaders['test'], args.device)
final_score = (test_acc["Acc"] + 1 - abs(mia["Forgetting Score"] * 2)) / 2
print(f'Test Acc: {test_acc}')
print(f'MIA: {mia}')
print(f'Final score: {final_score}')

NegGrad Epoch [1/1] Loss: 0.059
Test Acc: {'Loss': 0.019304277586314773, 'Acc': 0.7323989569752282, 'Top-2 Acc': 0.8715775749674055}
MIA: {'MIA Regression Accuracy': 0.5943168934240363, 'MIA CV Accuracy': 0.6466353677621284, 'Forgetting Score': 0.14663536776212838}
Final score: 0.7195641107254858


### UNSIR

In [27]:
from unlearn.unsir import Noise, generate_error_maximizing_noise, impair_step, repair_step
from torch.utils.data import DataLoader

class Args:
    def __init__(self):
        self.alpha = 0.1
        self.num_classes = 8
        self.device = torch.device(f"cuda:0" if torch.cuda.is_available() else "cpu")
        self.impair_epochs = 1
        self.repair_epochs = 2
        self.lr = 0.001

args = Args()
criterion = nn.CrossEntropyLoss()
model_path = 'checkpoint/last_100_emotion_original.pth'
unsir_model = load_model(7, args.device, model_path)

noise = generate_error_maximizing_noise(unsir_model, data_loaders['forget_train'], args.device, args.impair_epochs)

# Impair Step
noisy_data = [(noise()[i % noise().size(0)], data_loaders['forget_train'].dataset[i][1]) for i in range(len(data_loaders['forget_train'].dataset))]
noisy_loader = DataLoader(noisy_data, batch_size=128, shuffle=True)
impair_step(unsir_model, noisy_loader, args.device, args.lr, args.impair_epochs)

print("Performance of Impaired Model on Forget Samples")
test_acc = calculate_accuracy(unsir_model, data_loaders['test'], criterion, args.device)
mia_impaired = MIA(unsir_model, data_loaders["retain_test"], data_loaders['forget_test'], data_loaders['test'], args.device)
print(f'Test Acc: {test_acc}')
print(f'Impaired MIA : {mia_impaired}')

# Repair Step
repair_step(unsir_model, data_loaders['retain_train'], args.device, args.lr, args.repair_epochs)

print("Performance of Repaired Model on Forget Samples")
test_acc = calculate_accuracy(unsir_model, data_loaders['test'], criterion, args.device)
mia_repaired = MIA(unsir_model, data_loaders["retain_test"], data_loaders['forget_test'], data_loaders['test'], args.device)
final_score = (test_acc["Acc"] + 1 - abs(mia_repaired["Forgetting Score"] * 2)) / 2
print(f'Test Acc: {test_acc}')
print(f'Repaired MIA: {mia_repaired}')
print(f'Final score: {final_score}')

Epoch [1/1] - Noise Loss: 2921.9857
Impair Step - Train loss 1: 2.3708, Train Acc: 34.76%
Performance of Impaired Model on Forget Samples
Test Acc: {'Loss': 0.025897050618814522, 'Acc': 0.386245110821382, 'Top-2 Acc': 0.6078878748370273}
Impaired MIA : {'MIA Regression Accuracy': 0.5477607709750567, 'MIA CV Accuracy': 0.5195618153364631, 'Forgetting Score': 0.019561815336463062}
Repair Step - Train loss 1: 1.3011, Train Acc: 51.89%
Repair Step - Train loss 2: 2.3603, Train Acc: 115.01%
Performance of Repaired Model on Forget Samples
Test Acc: {'Loss': 0.016864737833338892, 'Acc': 0.6215775749674055, 'Top-2 Acc': 0.7920469361147328}
Repaired MIA: {'MIA Regression Accuracy': 0.5479024943310657, 'MIA CV Accuracy': 0.5195618153364631, 'Forgetting Score': 0.019561815336463062}
Final score: 0.7912269721472397


### BadT

In [29]:
from unlearn.badT import badt

class Args:
    def __init__(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.seed = 42
        self.bt_optim = "adam"
        self.bt_alpha = 1
        self.bt_beta = 1
        self.bt_kd_T = 4
        self.bt_epochs = 1
        self.bt_learning_rate = 0.001
        self.bt_lr_decay_epochs = [10, 10, 10]
        self.bt_lr_decay_rate = 0.1
        self.bt_weight_decay = 5e-4
        self.bt_momentum = 0.9

args = Args()
criterion = nn.CrossEntropyLoss()
device = torch.device("cuda:0" if torch.cuda.is_available() else "CPU")
model_path = 'checkpoint/last_100_emotion_original.pth'
gteacher = load_model(7, args.device, model_path)
bteacher = load_model(7, args.device, model_path=None)
student = load_model(7, args.device, model_path)

badT_model = badt(
    gteacher=gteacher,
    bteacher=bteacher,
    student=student,
    retain_loader=data_loaders["retain_train"],
    forget_loader=data_loaders["forget_train"],
    valid_loader_full=data_loaders["test"],
    args=args
)

test_acc = calculate_accuracy(badT_model, data_loaders['test'], criterion, args.device)
mia = MIA(badT_model, data_loaders["retain_test"], data_loaders['forget_test'], data_loaders['test'], args.device)
final_score = (test_acc["Acc"] + 1 - abs(mia["Forgetting Score"] * 2)) / 2
print(f'Test Acc: {test_acc}')
print(f'MIA: {mia}')
print(f'Final score: {final_score}')

==> Bad Teacher Unlearning ...
Epoch 1: train_acc: 53.60, train_loss: -55732.36
Test Acc: {'Loss': 340.42474463040304, 'Acc': 0.5153194263363755, 'Top-2 Acc': 0.6711212516297262}
MIA: {'MIA Regression Accuracy': 0.5477607709750567, 'MIA CV Accuracy': 0.5211267605633803, 'Forgetting Score': 0.021126760563380254}
Final score: 0.7365329526048074


### SCRUB

In [30]:
import torch
from unlearn.scrub import scrub

class Args:
    def __init__(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.seed = 42
        self.optim = "sgd"  
        self.sgda_epochs = 2
        self.sgda_learning_rate = 0.0005
        self.sgda_weight_decay = 5e-4
        self.sgda_momentum = 0.9
        self.msteps = 2
        self.kd_T = 4  
        self.lr_decay_epochs = [3, 5, 9]
        self.lr_decay_rate = 0.1

args = Args()
model_path =  'checkpoint/last_100_emotion_original.pth'
teacher_model = load_model(7, args.device, model_path)
student_model = load_model(7, args.device, model_path)

scrub_model = scrub(
    teacher=teacher_model,
    student=student_model,
    args=args,
    retain_loader_train=data_loaders["retain_train"],
    retain_loader_test=data_loaders["retain_test"],
    forget_loader_train=data_loaders["forget_train"],
    forget_loader_test=data_loaders["forget_test"],
    valid_loader_full=data_loaders["test"] 
)

test_acc = calculate_accuracy(scrub_model, data_loaders['test'], criterion, args.device)
mia = MIA(scrub_model, data_loaders["retain_test"], data_loaders['forget_test'], data_loaders['test'], args.device)
final_score = (test_acc["Acc"] + 1 - abs(mia["Forgetting Score"] * 2)) / 2
print(f'Test Acc: {test_acc}')
print(f'MIA: {mia}')
print(f'Final score: {final_score}')

==> scrub unlearning ...
Epoch 1: maximize loss: -0.12, train_acc: 98.80, minimize loss: 0.11
==> scrub unlearning ...
Epoch 2: maximize loss: -0.22, train_acc: 99.02, minimize loss: 0.11
Total time: 62.48 seconds
Test Acc: {'Loss': 0.018813876191098266, 'Acc': 0.750651890482399, 'Top-2 Acc': 0.878748370273794}
MIA: {'MIA Regression Accuracy': 0.6038832199546486, 'MIA CV Accuracy': 0.6375586854460094, 'Forgetting Score': 0.13755868544600935}
Final score: 0.7377672597951901


### DLFD

In [93]:
from unlearn.dlfd import train_dlfd, set_seed, linear_weight_scheduler
set_seed(42) 

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_path = f'checkpoint/last_100_emotion_original.pth'
dlfm_model = load_model(7, device, model_path)
path = 'checkpoint/emotion_dlfd.pth'

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(dlfm_model.parameters(), lr=0.006, momentum=0.9, weight_decay=5e-4)

forget_weight = 7  # Weight for forgetting
initial_retain_weight = 0.5  # Initial weight for retaining
final_retain_weight = 1.0  # Final weight for retaining
perturbation_steps = 20  # Number of steps for perturbation
perturbation_strength = 2.0  # Strength of perturbation
forget_threshold = 0.23  # Forgetting score threshold
best_test_acc = 0  # Save the best test accuracy
max_forgetting_score = 0.23  # Forgetting score saving threshold

train_dlfd(
        dlfm_model, data_loaders, criterion, optimizer, device,
        perturbation_steps, perturbation_strength, forget_threshold, max_forgetting_score,
        initial_retain_weight, final_retain_weight, forget_weight, path
    )

Starting Forgetting Phase...
Batch 1/121 - Test Acc: 0.7074, MIA Forgetting Score: 0.2826, Final Score: 0.5711
Batch 2/121 - Test Acc: 0.6968, MIA Forgetting Score: 0.2690, Final Score: 0.5794
Batch 3/121 - Test Acc: 0.6145, MIA Forgetting Score: 0.2475, Final Score: 0.5598
Batch 4/121 - Test Acc: 0.5884, MIA Forgetting Score: 0.2398, Final Score: 0.5544
Batch 5/121 - Test Acc: 0.5575, MIA Forgetting Score: 0.2396, Final Score: 0.5392
Batch 6/121 - Test Acc: 0.5460, MIA Forgetting Score: 0.2332, Final Score: 0.5398
Batch 7/121 - Test Acc: 0.6349, MIA Forgetting Score: 0.2382, Final Score: 0.5792
Batch 8/121 - Test Acc: 0.5974, MIA Forgetting Score: 0.2367, Final Score: 0.5620
Batch 9/121 - Test Acc: 0.5273, MIA Forgetting Score: 0.2336, Final Score: 0.5300
Batch 10/121 - Test Acc: 0.5436, MIA Forgetting Score: 0.2341, Final Score: 0.5377
Batch 11/121 - Test Acc: 0.5778, MIA Forgetting Score: 0.2297, Final Score: 0.5592
New best model saved with Test Acc: 0.5778
Forgetting score below t

In [94]:
model_path = 'checkpoint/emotion_dlfd.pth'
dlfm_model = load_model(7, device, model_path)

test_acc = calculate_accuracy(dlfm_model, data_loaders['test'], criterion, device)
mia = MIA(dlfm_model, data_loaders["retain_test"], data_loaders['forget_test'], data_loaders['test'], device)
final_score = (test_acc["Acc"] + 1 - abs(mia["Forgetting Score"] * 2)) / 2
print(f'Test Acc: {test_acc}')
print(f'MIA :{mia}')
print(f'Final score: {final_score}')

Test Acc: {'Loss': 0.015832779987325414, 'Acc': 0.6606910039113429, 'Top-2 Acc': 0.809973924380704}
MIA :{'MIA Regression Accuracy': 0.5508786848072562, 'MIA CV Accuracy': 0.5464788732394367, 'Forgetting Score': 0.04647887323943667}
Final score: 0.7838666287162348
